# 필요 라이브러리 불러오기

In [ ]:
# !pip install pytube

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles import Alignment
from pytube import YouTube
import datetime
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import time
import re
import requests
import random

c:\anaconda3\envs\py_397\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\anaconda3\envs\py_397\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\anaconda3\envs\py_397\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# 필요 클래스 정의

In [2]:
# 좀 더 빨리 스크롤 내리는 방법 인용 # https://manyda.tistory.com/10
#Queue의 기본적인 기능 구현
class Queue():
    def __init__(self, maxsize):
        self.queue = []
        self.maxsize = maxsize
        
    # Queue에 Data 넣음
    def enqueue(self, data):
        self.queue.append(data)

    # Queue에 가장 먼저 들어온 Data 내보냄
    def dequeue(self):
        dequeue_object = None
        if self.isEmpty():
            print("Queue is Empty")
        else:
            dequeue_object = self.queue[0]
            self.queue = self.queue[1:]
        return dequeue_object
    
    # Queue에 가장 먼저들어온 Data return
    def peek(self):
        peek_object = None
        if self.isEmpty():
            print("Queue is Empty")
        else:
            peek_object = self.queue[0]
        return peek_object
    
    # Queue가 비어있는지 확인
    def isEmpty(self):
        is_empty = False
        if len(self.queue) == 0:
            is_empty = True
        return is_empty
    
    # Queue의 Size가 Max Size를 초과하는지 확인
    def isMaxSizeOver(self):
        queue_size = len(self.queue)
        if (queue_size > self.maxsize):
            return False
        else :
            return True

# 유튜버 영상들 정보 수집

## 특정 유튜버 채널 들어가기

In [8]:
df = pd.DataFrame(columns=['제목', 'URL', '조회수', '업데이트 날짜', '영상 길이', '영상 좋아요 수']) # 저장 데이터프레임 생성

# 크롬 웹브라우저 실행
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome("chromedriver-win64/chromedriver.exe")
driver.maximize_window()

keyword = '회사원A' # 검색하고자 하는 유튜버 입력

# 키워드로 접속
driver.get(f"https://www.youtube.com/results?search_query={keyword}")
# time.sleep(2) # 오류나면 잠시 기다리는 코드 추가해보기
driver.find_element(By.XPATH, '//*[@id="avatar-section"]/a').click() # 채널 클릭
time.sleep(1)
driver.find_element(By.XPATH, '//*[@id="tabsContent"]/tp-yt-paper-tab[2]').click() # 동영상 목록 클릭

C:\Users\Mgyu\AppData\Local\Temp\ipykernel_25816\266153786.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver-win64/chromedriver.exe")


## 스크롤 내리기 

In [9]:
#down the scroll
body = driver.find_element(By.TAG_NAME, 'body')
last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

# max size 50의 Queue 생성
# 0.1sec * 50 = 5sec 동안 Scroll 업데이트가 없으면 스크롤 내리기 종료
szQ = Queue(50)
enqueue_count = 0

i = 0
while True:
    # Scroll 내리기
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    
    # Scroll Height를 가져오는 주기
    time.sleep(0.1)
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    # Queue가 꽉 차는 경우 스크롤 내리기 종료
    if(enqueue_count > szQ.maxsize):
        break
    
    # 첫 Loop 수행 (Queue가 비어있는 경우) 예외 처리
    if(szQ.isEmpty()) :
        szQ.enqueue(new_page_height)
        enqueue_count += 1
        
    # Queue에 가장 먼저 들어온 데이터와 새로 업데이트 된 Scroll Height를 비교함
    # 같으면 그대로 Enqueue, 다르면 Queue의 모든 Data를 Dequeue 후 새로운 Scroll Height를 Enqueue 함.    
    else :
        if(szQ.peek() == new_page_height) :
            szQ.enqueue(new_page_height)
            enqueue_count += 1
        else :
            szQ.enqueue(new_page_height)
            for z in range(enqueue_count) :
                szQ.dequeue()
            enqueue_count = 1
    
    # 이 부분은 모든 영상을 수집하기에는 무리가 있어서 추가한 부분 20을 더 작은 수나 더 큰 수로 바꿔서 실행해도 문제없음
    if i > 20:
        break
    else:
        i += 1

## 채널의 영상 정보들 수집

In [10]:
titles = driver.find_elements(By.XPATH, '//*[@id="video-title-link"]') 
for title in titles:
    main_title = title.get_property("title")
    url = title.get_property("href")
    df.loc[len(df), ['제목', 'URL']] =  [main_title, url]

In [11]:
df

,제목,URL,조회수,업데이트 날짜,영상 길이,영상 좋아요 수
0,10년째 유튜브하면서 요즘하고 있는 고민..😔같이 들어줄래?,https://www.youtube.com/watch?v=bcg4GqJ7Wok,NaN,NaN,NaN,NaN
1,✨나 떠히쓰 신인 아이돌인디? (feat.웨이크메이크),https://www.youtube.com/watch?v=Ig4Nj-Yk6uE,NaN,NaN,NaN,NaN
2,👙브라자에 천 만원 쓴 여자의 속옷리뷰 (feat.큰 컵 드루와)🩱,https://www.youtube.com/watch?v=p64X-KSe2DY,NaN,NaN,NaN,NaN
3,총 12kg 감량한 핫걸 지망생인데여...💚💙 바디프로필 찍었습니다😎,https://www.youtube.com/watch?v=ZUXbkhXWL1E,NaN,NaN,NaN,NaN
4,시부야 찐갸루✌언니한테 K-메이크업 끼얹으니🎀러블리즈 멤버가 됐는뎁쇼...?🥹😲(에...,https://www.youtube.com/watch?v=GhL3QKg0rBE,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
115,🇫🇮 사우나 근본국에선 관람차가 사우나라굽쇼??,https://www.youtube.com/watch?v=tLVmN2-tVsM,NaN,NaN,NaN,NaN
116,각종 공주복 장만 했습니다👗짐머만 산드로 미우미우 로로피아나 돌체앤가바나 포멜라토,https://www.youtube.com/watch?v=ZBXkJVVlMKc,NaN,NaN,NaN,NaN
117,리얼다이아몬드 파격가 31만원에 장만 하는 방법,https://www.youtube.com/watch?v=QhMniJwS6fU,NaN,NaN,NaN,NaN
118,시술 노노~ 셀프 괄사 마사지로 얼굴 땅콩핏 없애기??? 💜바이오힐보 광고,https://www.youtube.com/watch?v=YFTiDXSkS-g,NaN,NaN,NaN,NaN


## 라이브러리를 이용한 영상정보 수집

In [12]:
import tqdm


for i in tqdm.tqdm(range(len(df))):

    url = df.iloc[i,1]
    tube = YouTube(url)

    view = tube.views # 조회수 불러오기
    update_dates = str(tube.publish_date)  # 업데이트 날짜 불러와서 str 형태로 변환
    update_date = update_dates.split(" ")  # 업데이트 날짜 string을 공백으로 분리

    # 업데이트 날짜는 0000-00-00 00-00-00 형태인데 앞의 날짜부분만 추출하려 합니다.
    length_second = int(tube.length) # 유튜브 영상의 길이를 불러옴(문자 형태인것 같아서 int로 변환) / 문자가 아닐 수도...
    length = str(datetime.timedelta(seconds=length_second)) # datetime 모듈의 timedelta 함수로 초를 시:분:초 로 변환
    
    df.iloc[i, 2] = view 
    df.iloc[i, 3] = update_date[0] # 추출한 날짜 중 앞의 날짜부분만 추출
    df.iloc[i, 4] = length
df

100%|██████████| 120/120 [03:51<00:00,  1.93s/it]


,제목,URL,조회수,업데이트 날짜,영상 길이,영상 좋아요 수
0,10년째 유튜브하면서 요즘하고 있는 고민..😔같이 들어줄래?,https://www.youtube.com/watch?v=bcg4GqJ7Wok,131258,2023-09-09,0:11:17,NaN
1,✨나 떠히쓰 신인 아이돌인디? (feat.웨이크메이크),https://www.youtube.com/watch?v=Ig4Nj-Yk6uE,65571,2023-09-05,0:12:14,NaN
2,👙브라자에 천 만원 쓴 여자의 속옷리뷰 (feat.큰 컵 드루와)🩱,https://www.youtube.com/watch?v=p64X-KSe2DY,271352,2023-09-02,0:16:14,NaN
3,총 12kg 감량한 핫걸 지망생인데여...💚💙 바디프로필 찍었습니다😎,https://www.youtube.com/watch?v=ZUXbkhXWL1E,190373,2023-08-30,0:22:08,NaN
4,시부야 찐갸루✌언니한테 K-메이크업 끼얹으니🎀러블리즈 멤버가 됐는뎁쇼...?🥹😲(에...,https://www.youtube.com/watch?v=GhL3QKg0rBE,189224,2023-08-25,0:11:17,NaN
...,...,...,...,...,...,...
115,🇫🇮 사우나 근본국에선 관람차가 사우나라굽쇼??,https://www.youtube.com/watch?v=tLVmN2-tVsM,260365,2022-05-21,0:14:17,NaN
116,각종 공주복 장만 했습니다👗짐머만 산드로 미우미우 로로피아나 돌체앤가바나 포멜라토,https://www.youtube.com/watch?v=ZBXkJVVlMKc,273892,2022-05-15,0:12:39,NaN
117,리얼다이아몬드 파격가 31만원에 장만 하는 방법,https://www.youtube.com/watch?v=QhMniJwS6fU,211356,2022-05-14,0:12:12,NaN
118,시술 노노~ 셀프 괄사 마사지로 얼굴 땅콩핏 없애기??? 💜바이오힐보 광고,https://www.youtube.com/watch?v=YFTiDXSkS-g,126456,2022-05-12,0:14:58,NaN


In [16]:
df = df[df['업데이트 날짜'] > '2022-08-01']
df

,제목,URL,조회수,업데이트 날짜,영상 길이,영상 좋아요 수
0,10년째 유튜브하면서 요즘하고 있는 고민..😔같이 들어줄래?,https://www.youtube.com/watch?v=bcg4GqJ7Wok,131258,2023-09-09,0:11:17,NaN
1,✨나 떠히쓰 신인 아이돌인디? (feat.웨이크메이크),https://www.youtube.com/watch?v=Ig4Nj-Yk6uE,65571,2023-09-05,0:12:14,NaN
2,👙브라자에 천 만원 쓴 여자의 속옷리뷰 (feat.큰 컵 드루와)🩱,https://www.youtube.com/watch?v=p64X-KSe2DY,271352,2023-09-02,0:16:14,NaN
3,총 12kg 감량한 핫걸 지망생인데여...💚💙 바디프로필 찍었습니다😎,https://www.youtube.com/watch?v=ZUXbkhXWL1E,190373,2023-08-30,0:22:08,NaN
4,시부야 찐갸루✌언니한테 K-메이크업 끼얹으니🎀러블리즈 멤버가 됐는뎁쇼...?🥹😲(에...,https://www.youtube.com/watch?v=GhL3QKg0rBE,189224,2023-08-25,0:11:17,NaN
...,...,...,...,...,...,...
92,인생 최대 피부 뒤집어짐 : 피부 사막화 됐는데요... 홈케어로 어떻게든 되돌려 보...,https://www.youtube.com/watch?v=o-5ParZR7d4,151422,2022-08-20,0:12:19,NaN
93,🇺🇲미국에서 사 온 짐머만 그리고 프라다 미우미우,https://www.youtube.com/watch?v=MxhkfpvnjVo,334243,2022-08-14,0:14:06,NaN
94,🇺🇲미국 억만장자 화장품회사 사장이 지은 지하벙커에 사우나 수영장이 있다고??,https://www.youtube.com/watch?v=LGh_nG36-Ss,246206,2022-08-13,0:10:34,NaN
95,"🇺🇲붕대 감고있으면 살빠진대서 가 봤는데요, 150만원...🤦‍♀️",https://www.youtube.com/watch?v=cptBoRDcmXQ,392123,2022-08-07,0:11:28,NaN


In [13]:
# https://pytube.io/
print("제목 : ", tube.title)
print("길이 : ", tube.length)
print("게시자 : ", tube.author)
print("게시날짜 : ", tube.publish_date)
print("조회수 : ", tube.views)
print("키워드 : ", tube.keywords)
print("설명 : ", tube.description)
print("썸네일 : ", tube.thumbnail_url)

제목 :  🇺🇸미국 미용실에서 횃불로 머리 잘라주는데요...100만원...🙊
길이 :  599
게시자 :  회사원A
게시날짜 :  2022-05-08 00:00:00
조회수 :  292036
키워드 :  []
설명 :  None
썸네일 :  https://i.ytimg.com/vi/ngXPFveJAfM/hq720.jpg


In [18]:
driver = webdriver.Chrome("chromedriver-win64/chromedriver.exe")

C:\Users\Mgyu\AppData\Local\Temp\ipykernel_16336\93324219.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver-win64/chromedriver.exe")


# 특정 영상의 댓글 수집

In [19]:
# #set option of selenium
# options = webdriver.ChromeOptions()
# options.add_argument('window-size=1920x1080')
# options.add_argument('disable-gpu')
# options.add_argument('user')
# options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
# options.add_argument("lang=ko_KR")
# # driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
# driver = webdriver.Chrome("chromedriver-win64/chromedriver.exe")

import os
# 수집한 영상들에 대해서 반복해서 댓글 수집
for video_num in range(len(df)):
    url = df.iloc[video_num+2,1]
    driver.get(url)

    #페이지 Open 후 기다리는 시간
    time.sleep(7)
    
    # 영상 좋아요 수
    temp = driver.find_element(By.XPATH, '//*[@id="segmented-like-button"]/ytd-toggle-button-renderer/yt-button-shape/button')
    good = temp.get_attribute('aria-label')
    good_num = re.findall(r'\d{1,3}(?:,\d{3})*', good)
    if good_num:
        good_num = good_num[0].replace(',', '')  # 쉼표 제거
        df.iloc[video_num, 5] = good_num



    #down the scroll
    body = driver.find_element(By.TAG_NAME, 'body')
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    # max size 50의 Queue 생성
    # 0.1sec * 50 = 5sec 동안 Scroll 업데이트가 없으면 스크롤 내리기 종료
    szQ = Queue(3)
    enqueue_count = 0
    time.sleep(5)
    i = 0
    while True:
        # Scroll 내리기
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight / 10);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight / 2);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") 
        # Scroll Height를 가져오는 주기
        time.sleep(0.1)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        # Queue가 꽉 차는 경우 스크롤 내리기 종료
        if(enqueue_count > szQ.maxsize):
            break
        
        # 첫 Loop 수행 (Queue가 비어있는 경우) 예외 처리
        if(szQ.isEmpty()) :
            szQ.enqueue(new_page_height)
            enqueue_count += 1
            
        # Queue에 가장 먼저 들어온 데이터와 새로 업데이트 된 Scroll Height를 비교함
        # 같으면 그대로 Enqueue, 다르면 Queue의 모든 Data를 Dequeue 후 새로운 Scroll Height를 Enqueue 함.    
        else :
            if(szQ.peek() == new_page_height) :
                szQ.enqueue(new_page_height)
                enqueue_count += 1
            else :
                szQ.enqueue(new_page_height)
                for z in range(enqueue_count) :
                    szQ.dequeue()
                enqueue_count = 1
        time.sleep(5)
        if i > 30:
            break
        else:
            i += 1


    time.sleep(1.5)
           
    print('스크롤 끝')
    time.sleep(5)   
    # 댓글 정보 불러오기
    comments_list = driver.find_elements(By.XPATH, '//*[@id="comment"]')

    comment_data = pd.DataFrame({'youtube_id':[],
                                'comment':[],
                                'like_num':[], 'comment_date':[]})
    html_s0 = driver.page_source
    html_s = BeautifulSoup(html_s0,'html.parser')

    comment0 = html_s.find_all('ytd-comment-renderer',{'class':'style-scope ytd-comment-thread-renderer'})
    print('댓글정보 불러오기 시작')
    for i in range(len(comment0)):
        

        # 댓글
        comment = comment0[i].find('yt-formatted-string',{'id':'content-text','class':'style-scope ytd-comment-renderer'}).text
        # 댓글의 좋아요 수
        try:
            aa = comment0[i].find('span',{'id':'vote-count-left'}).text
            #정규표현식으로 숫자만 추출하는 것은 정규표현식에 대한 공부를 더 한 뒤 해결
            #re.findall('[0-9]',aa)
            #"".join(re.findall('[0-9]',aa)) -> 리스트 내부의 문자열의 합
            like_num = "".join(re.findall('[0-9]',aa))
            like_num = int(like_num)
        except:
            like_num = 0

        # 댓글 작성자 id    
        bb = comment0[i].find('a',{'id':'author-text'}).find('span').text
        youtube_id = "".join(re.findall('[가-힣0-9a-zA-Z]',bb))

        # 댓글 작성 날짜
        comment_date = comment0[i].find('yt-formatted-string',{'class':'published-time-text style-scope ytd-comment-renderer'}).text
        
        # 데이터프레임에 데이터 저장
        comment_data.loc[len(comment_data), :] = [youtube_id, comment, like_num, comment_date]

    comment_data.index = range(len(comment_data))
    # 파일 저장
    if not os.path.exists(keyword):
        os.makedirs(keyword)

    comment_data.to_csv(f'./data/{keyword}/{keyword}_{video_num}_comment_data.csv')
    print(f'{keyword}_{video_num}_comment_data.csv 저장완료')
    

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=116.0.5845.188)
Stacktrace:
	GetHandleVerifier [0x00007FF7D1CA52A2+57122]
	(No symbol) [0x00007FF7D1C1EA92]
	(No symbol) [0x00007FF7D1AEE3AB]
	(No symbol) [0x00007FF7D1ACE1B9]
	(No symbol) [0x00007FF7D1B4B417]
	(No symbol) [0x00007FF7D1B5E24F]
	(No symbol) [0x00007FF7D1B46DB3]
	(No symbol) [0x00007FF7D1B1D2B1]
	(No symbol) [0x00007FF7D1B1E494]
	GetHandleVerifier [0x00007FF7D1F4EF82+2849794]
	GetHandleVerifier [0x00007FF7D1FA1D24+3189156]
	GetHandleVerifier [0x00007FF7D1F9ACAF+3160367]
	GetHandleVerifier [0x00007FF7D1D36D06+653702]
	(No symbol) [0x00007FF7D1C2A208]
	(No symbol) [0x00007FF7D1C262C4]
	(No symbol) [0x00007FF7D1C263F6]
	(No symbol) [0x00007FF7D1C167A3]
	BaseThreadInitThunk [0x00007FFC2E757614+20]
	RtlUserThreadStart [0x00007FFC304026B1+33]


In [35]:
comment_data

,youtube_id,comment,like_num,comment_date
0,happyfeliz9014,여러모로 불편한 점 때문에 더 좋은 이름이 있으면 바꾸는게 좋지만 갠적으로는 회사원...,573.0,1일 전
1,SYjb1ur,그냥 쉬어도 되는데 세이브 영상 까지 \n준비해놓고 간다는 울 언니…\n성공한 여자...,520.0,1일 전
2,yurihada,저는 쇼호스트다 보니 수년전부터 회사원님 영상 보면서 시장조사를 했고 지금은 유튜버...,147.0,1일 전
3,trom11111100,회사원님의 휴가 너무 응원하고 즐거운 시간 보내시길 바랄게요!\n활동명을 바꾸는 이...,184.0,1일 전
4,Yourkristin,많은 유튜버들이 있지만 회사원a님 컨텐츠는 쉬운 이른바 쉬어가는 컨텐츠가 없고 남이...,195.0,1일 전
...,...,...,...,...
153,yy1996,언니 .. 햇빛 아래서? 찍으서서 그런지 뿌연 느낌이 강해서 ㅜㅜ 보기 조금 불편한...,0.0,23시간 전
154,sheiskal,더희 어때융? The hee !,3.0,1일 전
155,nukki2869,A-Sawon 에이사원은요 ? ㅋㅋ,0.0,10시간 전
156,usersf9rw8lc3r,고민이 있으면 눈 딱 감고 결혼하세요.,0.0,1일 전


# 전체 반복문

In [22]:
temp_list = pd.read_excel('./data/뷰티_리스트.xlsx')
temp_list.iloc[197:200, 0].to_list()    

['그레인Grain', '문군MOONGOON', '보니bonny']

In [23]:
temp_list.iloc[197:200, 0].to_list() + ['레삐디오(RepitProfessional)', '몽키TV', '패션-정찬양']

['그레인Grain',
 '문군MOONGOON',
 '보니bonny',
 '레삐디오(RepitProfessional)',
 '몽키TV',
 '패션-정찬양']

In [ ]:
temp_list.iloc[180:200, 0].to_list() 

In [25]:
import os
youtuber_list = temp_list.iloc[197:200, 0].to_list() + ['레삐디오(RepitProfessional)', '몽키TV', '패션-정찬양']


# 크롬 웹브라우저 실행
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome("chromedriver-win64/chromedriver.exe")
driver.maximize_window()

for youtuber in youtuber_list:
    df = pd.DataFrame(columns=['제목', 'URL', '조회수', '업데이트 날짜', '영상 길이', '영상 좋아요 수']) # 저장 데이터프레임 생성


    # 키워드로 접속
    driver.get(f"https://www.youtube.com/results?search_query={youtuber}")
    # time.sleep(2) # 오류나면 잠시 기다리는 코드 추가해보기
    driver.find_element(By.XPATH, '//*[@id="avatar-section"]/a').click() # 채널 클릭
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="tabsContent"]/tp-yt-paper-tab[2]').click() # 동영상 목록 클릭
    


    #down the scroll
    body = driver.find_element(By.TAG_NAME, 'body')
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    # max size 50의 Queue 생성
    # 0.1sec * 50 = 5sec 동안 Scroll 업데이트가 없으면 스크롤 내리기 종료
    szQ = Queue(50)
    enqueue_count = 0

    i = 0
    while True:
        # Scroll 내리기
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        
        # Scroll Height를 가져오는 주기
        time.sleep(0.1)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        # Queue가 꽉 차는 경우 스크롤 내리기 종료
        if(enqueue_count > szQ.maxsize):
            break
        
        # 첫 Loop 수행 (Queue가 비어있는 경우) 예외 처리
        if(szQ.isEmpty()) :
            szQ.enqueue(new_page_height)
            enqueue_count += 1
            
        # Queue에 가장 먼저 들어온 데이터와 새로 업데이트 된 Scroll Height를 비교함
        # 같으면 그대로 Enqueue, 다르면 Queue의 모든 Data를 Dequeue 후 새로운 Scroll Height를 Enqueue 함.    
        else :
            if(szQ.peek() == new_page_height) :
                szQ.enqueue(new_page_height)
                enqueue_count += 1
            else :
                szQ.enqueue(new_page_height)
                for z in range(enqueue_count) :
                    szQ.dequeue()
                enqueue_count = 1
        
        # 이 부분은 모든 영상을 수집하기에는 무리가 있어서 추가한 부분 20을 더 작은 수나 더 큰 수로 바꿔서 실행해도 문제없음
        print(i)
        if i > 6:
            break
        else:
            i += 1

    titles = driver.find_elements(By.XPATH, '//*[@id="video-title-link"]') 
    for title in titles:
        main_title = title.get_property("title")
        url = title.get_property("href")
        df.loc[len(df), ['제목', 'URL']] =  [main_title, url]

    
    for i in range(len(df)):

        url = df.iloc[i,1]
        tube = YouTube(url)

        view = tube.views # 조회수 불러오기
        update_dates = str(tube.publish_date)  # 업데이트 날짜 불러와서 str 형태로 변환
        update_date = update_dates.split(" ")  # 업데이트 날짜 string을 공백으로 분리

        # 업데이트 날짜는 0000-00-00 00-00-00 형태인데 앞의 날짜부분만 추출하려 합니다.
        length_second = int(tube.length) # 유튜브 영상의 길이를 불러옴(문자 형태인것 같아서 int로 변환) / 문자가 아닐 수도...
        length = str(datetime.timedelta(seconds=length_second)) # datetime 모듈의 timedelta 함수로 초를 시:분:초 로 변환
        
        df.iloc[i, 2] = view 
        df.iloc[i, 3] = update_date[0] # 추출한 날짜 중 앞의 날짜부분만 추출
        df.iloc[i, 4] = length
        if update_date[0] < '2022-08-01':
            break

    df = df[df['업데이트 날짜'] > '2022-08-01']
    df = df[df['업데이트 날짜'] < '2023-09-01']
    try:
        os.makedirs(f'./data/{youtuber}')
    except:
        pass
    df.to_csv(f'./data/{youtuber}/{youtuber}_video_data.csv')
    print(f'{youtuber}_video_data.csv 저장완료')
    


    import os
    # 수집한 영상들에 대해서 반복해서 댓글 수집
    for video_num in range(len(df)):
        url = df.iloc[video_num,1]
        driver.get(url)

        #페이지 Open 후 기다리는 시간
        time.sleep(5)
        
        # 영상 좋아요 수
        temp = driver.find_element(By.XPATH, '//*[@id="segmented-like-button"]/ytd-toggle-button-renderer/yt-button-shape/button')
        good = temp.get_attribute('aria-label')
        good_num = re.findall(r'\d{1,3}(?:,\d{3})*', good)
        if good_num:
            good_num = good_num[0].replace(',', '')  # 쉼표 제거
            df.iloc[video_num, 5] = good_num



        #down the scroll
        body = driver.find_element(By.TAG_NAME, 'body')
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        # max size 50의 Queue 생성
        # 0.1sec * 50 = 5sec 동안 Scroll 업데이트가 없으면 스크롤 내리기 종료
        szQ = Queue(3)
        enqueue_count = 0
        time.sleep(5)
        i = 0
        while True:
            # Scroll 내리기
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight / 10);")
            time.sleep(1)
            # driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight / 2);")
            # time.sleep(1)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") 
            # Scroll Height를 가져오는 주기
            time.sleep(0.1)
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            
            # Queue가 꽉 차는 경우 스크롤 내리기 종료
            if(enqueue_count > szQ.maxsize):
                break
            
            # 첫 Loop 수행 (Queue가 비어있는 경우) 예외 처리
            if(szQ.isEmpty()) :
                szQ.enqueue(new_page_height)
                enqueue_count += 1
                
            # Queue에 가장 먼저 들어온 데이터와 새로 업데이트 된 Scroll Height를 비교함
            # 같으면 그대로 Enqueue, 다르면 Queue의 모든 Data를 Dequeue 후 새로운 Scroll Height를 Enqueue 함.    
            else :
                if(szQ.peek() == new_page_height) :
                    szQ.enqueue(new_page_height)
                    enqueue_count += 1
                else :
                    szQ.enqueue(new_page_height)
                    for z in range(enqueue_count) :
                        szQ.dequeue()
                    enqueue_count = 1
            time.sleep(2)
            if i > 10:
                break
            else:
                print(i)
                i += 1
                   
        print('스크롤 끝')
        time.sleep(3)   
        # 댓글 정보 불러오기
        comments_list = driver.find_elements(By.XPATH, '//*[@id="comment"]')

        comment_data = pd.DataFrame({'youtube_id':[],
                                    'comment':[],
                                    'like_num':[], 'comment_date':[]})
        html_s0 = driver.page_source
        html_s = BeautifulSoup(html_s0,'html.parser')

        comment0 = html_s.find_all('ytd-comment-renderer',{'class':'style-scope ytd-comment-thread-renderer'})
        print('댓글정보 불러오기 시작')
        for i in range(len(comment0)):
            

            # 댓글
            comment = comment0[i].find('yt-formatted-string',{'id':'content-text','class':'style-scope ytd-comment-renderer'}).text
            # 댓글의 좋아요 수
            try:
                aa = comment0[i].find('span',{'id':'vote-count-left'}).text
                #정규표현식으로 숫자만 추출하는 것은 정규표현식에 대한 공부를 더 한 뒤 해결
                #re.findall('[0-9]',aa)
                #"".join(re.findall('[0-9]',aa)) -> 리스트 내부의 문자열의 합
                like_num = "".join(re.findall('[0-9]',aa))
                like_num = int(like_num)
            except:
                like_num = 0

            # 댓글 작성자 id    
            bb = comment0[i].find('a',{'id':'author-text'}).find('span').text
            youtube_id = "".join(re.findall('[가-힣0-9a-zA-Z]',bb))

            # 댓글 작성 날짜
            comment_date = comment0[i].find('yt-formatted-string',{'class':'published-time-text style-scope ytd-comment-renderer'}).text
            
            # 데이터프레임에 데이터 저장
            comment_data.loc[len(comment_data), :] = [youtube_id, comment, like_num, comment_date]

        comment_data.index = range(len(comment_data))
        # 파일 저장


        comment_data.to_csv(f'./data/{youtuber}/{youtuber}_{video_num}_comment_data.csv')
        print(f'{youtuber}_{video_num}_comment_data.csv 저장완료')
    df.to_csv(f'./data/{youtuber}/{youtuber}_video_data.csv')
    print(f'{youtuber}_video_data.csv 저장완료')
        

C:\Users\Mgyu\AppData\Local\Temp\ipykernel_23680\4073639125.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver-win64/chromedriver.exe")


0
1
2
3
4
5
6
7
그레인Grain_video_data.csv 저장완료
0
1
2
3
4
5
6
7
8
9
10
스크롤 끝
댓글정보 불러오기 시작
그레인Grain_0_comment_data.csv 저장완료
0
1
2
3
4
5
6
7
8
9
10
스크롤 끝
댓글정보 불러오기 시작
그레인Grain_1_comment_data.csv 저장완료
0
1
2
3
4
5
6
7
8
9
10
스크롤 끝
댓글정보 불러오기 시작
그레인Grain_2_comment_data.csv 저장완료
0
1
2
3
4
5
6
7
8
9
10
스크롤 끝
댓글정보 불러오기 시작
그레인Grain_3_comment_data.csv 저장완료
0
1
2
3
4
5
6
7
8
9
10
스크롤 끝
댓글정보 불러오기 시작
그레인Grain_4_comment_data.csv 저장완료
0
1
2
3
4
5
6
7
8
9
10
스크롤 끝
댓글정보 불러오기 시작
그레인Grain_5_comment_data.csv 저장완료
0
1
2
3
4
5
6
7
8
9
스크롤 끝
댓글정보 불러오기 시작
그레인Grain_6_comment_data.csv 저장완료
0
1
2
3
4
5
6
7
8
9
스크롤 끝
댓글정보 불러오기 시작
그레인Grain_7_comment_data.csv 저장완료
0
1
2
3
4
5
6
7
8
9
10
스크롤 끝
댓글정보 불러오기 시작
그레인Grain_8_comment_data.csv 저장완료
0
1
2
3
4
5
6
7
8
9
10
스크롤 끝
댓글정보 불러오기 시작
그레인Grain_9_comment_data.csv 저장완료
0
1
2
3
4
5
6
7
8
9
10
스크롤 끝
댓글정보 불러오기 시작
그레인Grain_10_comment_data.csv 저장완료
0
1
2
3
4
5
6
7
8
9
10
스크롤 끝
댓글정보 불러오기 시작
그레인Grain_11_comment_data.csv 저장완료
0
1
2
3
4
5
6
7
8
9
10
스크롤 끝
댓글정보 불러오기 시작
그레인Grain_12_comme

In [34]:
youtuber_list = ['회사원A', '씬님', '기우쌤'] 

for youtuber in youtuber_list:
    df = pd.DataFrame(columns=['제목', 'URL', '조회수', '업데이트 날짜', '영상 길이', '영상 좋아요 수']) # 저장 데이터프레임 생성

    # 크롬 웹브라우저 실행
    # driver = webdriver.Chrome(ChromeDriverManager().install())
    driver = webdriver.Chrome("chromedriver-win64/chromedriver.exe")
    driver.maximize_window()

    # 키워드로 접속
    driver.get(f"https://www.youtube.com/results?search_query={youtuber}")
    # time.sleep(2) # 오류나면 잠시 기다리는 코드 추가해보기
    driver.find_element(By.XPATH, '//*[@id="avatar-section"]/a').click() # 채널 클릭
    time.sleep(3)
    driver.find_element(By.XPATH, '//*[@id="tabsContent"]/tp-yt-paper-tab[2]').click() # 동영상 목록 클릭
    


    #down the scroll
    body = driver.find_element(By.TAG_NAME, 'body')
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    # max size 50의 Queue 생성
    # 0.1sec * 50 = 5sec 동안 Scroll 업데이트가 없으면 스크롤 내리기 종료
    szQ = Queue(50)
    enqueue_count = 0
    time.sleep(3)
    i = 0
    while True:
        # Scroll 내리기
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        
        # Scroll Height를 가져오는 주기
        time.sleep(0.1)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        # Queue가 꽉 차는 경우 스크롤 내리기 종료
        if(enqueue_count > szQ.maxsize):
            break
        
        # 첫 Loop 수행 (Queue가 비어있는 경우) 예외 처리
        if(szQ.isEmpty()) :
            szQ.enqueue(new_page_height)
            enqueue_count += 1
            
        # Queue에 가장 먼저 들어온 데이터와 새로 업데이트 된 Scroll Height를 비교함
        # 같으면 그대로 Enqueue, 다르면 Queue의 모든 Data를 Dequeue 후 새로운 Scroll Height를 Enqueue 함.    
        else :
            if(szQ.peek() == new_page_height) :
                szQ.enqueue(new_page_height)
                enqueue_count += 1
            else :
                szQ.enqueue(new_page_height)
                for z in range(enqueue_count) :
                    szQ.dequeue()
                enqueue_count = 1
        
        # 이 부분은 모든 영상을 수집하기에는 무리가 있어서 추가한 부분 20을 더 작은 수나 더 큰 수로 바꿔서 실행해도 문제없음
        print(i)
        if i > 6:
            break
        else:
            i += 1

    titles = driver.find_elements(By.XPATH, '//*[@id="video-title-link"]') 
    for title in titles:
        main_title = title.get_property("title")
        url = title.get_property("href")
        df.loc[len(df), ['제목', 'URL']] =  [main_title, url]

    
    for i in range(len(df)):

        url = df.iloc[i,1]
        tube = YouTube(url)

        view = tube.views # 조회수 불러오기
        update_dates = str(tube.publish_date)  # 업데이트 날짜 불러와서 str 형태로 변환
        update_date = update_dates.split(" ")  # 업데이트 날짜 string을 공백으로 분리

        # 업데이트 날짜는 0000-00-00 00-00-00 형태인데 앞의 날짜부분만 추출하려 합니다.
        length_second = int(tube.length) # 유튜브 영상의 길이를 불러옴(문자 형태인것 같아서 int로 변환) / 문자가 아닐 수도...
        length = str(datetime.timedelta(seconds=length_second)) # datetime 모듈의 timedelta 함수로 초를 시:분:초 로 변환
        
        df.iloc[i, 2] = view 
        df.iloc[i, 3] = update_date[0] # 추출한 날짜 중 앞의 날짜부분만 추출
        df.iloc[i, 4] = length
        if update_date[0] < '2022-08-01':
            break

    df = df[df['업데이트 날짜'] > '2022-08-01']
    df = df[df['업데이트 날짜'] < '2023-09-01']
    try:
        os.makedirs(f'./data/{youtuber}')
    except:
        pass
    df.to_csv(f'./data/{youtuber}/{youtuber}_video_data.csv')
    print(f'{youtuber}_video_data.csv 저장완료')
    


    import os
    # 수집한 영상들에 대해서 반복해서 댓글 수집
    for video_num in range(len(df)):
        url = df.iloc[video_num,1]
        driver.get(url)

        #페이지 Open 후 기다리는 시간
        time.sleep(5)
        
        # 영상 좋아요 수
        temp = driver.find_element(By.XPATH, '//*[@id="segmented-like-button"]/ytd-toggle-button-renderer/yt-button-shape/button')
        good = temp.get_attribute('aria-label')
        good_num = re.findall(r'\d{1,3}(?:,\d{3})*', good)
        if good_num:
            good_num = good_num[0].replace(',', '')  # 쉼표 제거
            df.iloc[video_num, 5] = good_num


        # comment_data.to_csv(f'./data/{youtuber}/{youtuber}_{video_num}_comment_data.csv')
        # print(f'{youtuber}_{video_num}_comment_data.csv 저장완료')
    df.to_csv(f'./data/{youtuber}/{youtuber}_video_data.csv')
    print(f'{youtuber}_video_data.csv 저장완료')
        

C:\Users\Mgyu\AppData\Local\Temp\ipykernel_16336\974230692.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver-win64/chromedriver.exe")


0
1
2
3
4
5
6
7
회사원A_video_data.csv 저장완료
회사원A_video_data.csv 저장완료


C:\Users\Mgyu\AppData\Local\Temp\ipykernel_16336\974230692.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver-win64/chromedriver.exe")


0
1
2
3
4
5
6
7
씬님_video_data.csv 저장완료
씬님_video_data.csv 저장완료


C:\Users\Mgyu\AppData\Local\Temp\ipykernel_16336\974230692.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver-win64/chromedriver.exe")


0
1
2
3
4
5
6
7
기우쌤_video_data.csv 저장완료
기우쌤_video_data.csv 저장완료


In [26]:
a = pd.read_csv('./data/씬님/씬님_video_data.csv')
a

,Unnamed: 0,제목,URL,조회수,업데이트 날짜,영상 길이,영상 좋아요 수
0,1,[ENG] 날 때부터 재벌 처럼 보이는 메이크업 #올드머니룩 ㅋㅋㅋㅋ,https://www.youtube.com/watch?v=Z8EKa-VA_cY,33027,2023-08-31,0:15:24,NaN
1,2,"[ENG] 브랜드썰 : 샬롯틸버리 편🔮about 마녀 음모론, 샬롯틸버리 추천템",https://www.youtube.com/watch?v=s02A3jp6-1A,44153,2023-08-23,0:18:21,NaN
2,3,[ENG]🤬절대 사면 안되는 화장품 7개 공개합니다 // 극혐 화장품 2탄? ㅋㅋㅋㅋ//,https://www.youtube.com/watch?v=s9jHu_qvZuc,524839,2023-08-16,0:12:44,NaN
3,4,"[ENG] 상여자의👨🏻 수염, 겨털 면도기 리뷰🧖🏻 여름철 왁싱 관리 용품 추천!",https://www.youtube.com/watch?v=Nt3JW7UWyNI,99473,2023-08-12,0:15:19,NaN
4,5,[ENG] 비교분석✍🏻 '요즘 MZ 스모키 vs 10년전 라떼 스모키' 메이크업 하는법,https://www.youtube.com/watch?v=xAZkixswn4E,102469,2023-08-09,0:18:18,NaN
5,6,"[ENG] 요즘 이 섀도우만 써요!! 여름철 모공관리 머드팩, 요즘 메이크업 루틴 ...",https://www.youtube.com/watch?v=7N0t31DPZwc,100815,2023-07-26,0:16:00,NaN
6,7,[ENG] 화장의 근본(根本) : 🎨색(色)의 원리 (1)이론 편,https://www.youtube.com/watch?v=0DHg-478pns,63010,2023-07-19,0:10:31,NaN
7,8,"[ENG]🩸말로만 설명하면 어떻게 알아 직쩝 껴봐야 알지 ?!!🩸[생리컵, 생리디스...",https://www.youtube.com/watch?v=8pfprJEiNFg,185017,2023-07-12,0:17:20,NaN
8,9,(3화) [ENG] 쿠션리뷰 번외편 : 저번에 빼먹은 여름 신상 5개 🪼재빠르게 리...,https://www.youtube.com/watch?v=K51wIXw8Duw,122980,2023-07-04,0:10:05,NaN
9,10,"[ENG] 파데는 그저 거들 뿐👍🏻얘네들 없으면 씬님도 화장 못해… 베이스, 프라이...",https://www.youtube.com/watch?v=cwBrWV5OATU,189889,2023-06-28,0:14:06,NaN


In [ ]:
# #set option of selenium
# options = webdriver.ChromeOptions()
# options.add_argument('window-size=1920x1080')
# options.add_argument('disable-gpu')
# options.add_argument('user')
# options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
# options.add_argument("lang=ko_KR")
# # driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
# driver = webdriver.Chrome("chromedriver-win64/chromedriver.exe")

import os
# 수집한 영상들에 대해서 반복해서 댓글 수집
for video_num in range(len(df)):
    url = df.iloc[video_num+2,1]
    driver.get(url)

    #페이지 Open 후 기다리는 시간
    time.sleep(7)
    
    # 영상 좋아요 수
    temp = driver.find_element(By.XPATH, '//*[@id="segmented-like-button"]/ytd-toggle-button-renderer/yt-button-shape/button')
    good = temp.get_attribute('aria-label')
    good_num = re.findall(r'\d{1,3}(?:,\d{3})*', good)
    if good_num:
        good_num = good_num[0].replace(',', '')  # 쉼표 제거
        df.iloc[video_num, 5] = good_num



    #down the scroll
    body = driver.find_element(By.TAG_NAME, 'body')
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    # max size 50의 Queue 생성
    # 0.1sec * 50 = 5sec 동안 Scroll 업데이트가 없으면 스크롤 내리기 종료
    szQ = Queue(3)
    enqueue_count = 0
    time.sleep(5)
    i = 0
    while True:
        # Scroll 내리기
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight / 10);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight / 2);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") 
        # Scroll Height를 가져오는 주기
        time.sleep(0.1)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        # Queue가 꽉 차는 경우 스크롤 내리기 종료
        if(enqueue_count > szQ.maxsize):
            break
        
        # 첫 Loop 수행 (Queue가 비어있는 경우) 예외 처리
        if(szQ.isEmpty()) :
            szQ.enqueue(new_page_height)
            enqueue_count += 1
            
        # Queue에 가장 먼저 들어온 데이터와 새로 업데이트 된 Scroll Height를 비교함
        # 같으면 그대로 Enqueue, 다르면 Queue의 모든 Data를 Dequeue 후 새로운 Scroll Height를 Enqueue 함.    
        else :
            if(szQ.peek() == new_page_height) :
                szQ.enqueue(new_page_height)
                enqueue_count += 1
            else :
                szQ.enqueue(new_page_height)
                for z in range(enqueue_count) :
                    szQ.dequeue()
                enqueue_count = 1
        time.sleep(5)
        if i > 30:
            break
        else:
            i += 1


    time.sleep(1.5)
           
    print('스크롤 끝')
    time.sleep(5)   
    # 댓글 정보 불러오기
    comments_list = driver.find_elements(By.XPATH, '//*[@id="comment"]')

    comment_data = pd.DataFrame({'youtube_id':[],
                                'comment':[],
                                'like_num':[], 'comment_date':[]})
    html_s0 = driver.page_source
    html_s = BeautifulSoup(html_s0,'html.parser')

    comment0 = html_s.find_all('ytd-comment-renderer',{'class':'style-scope ytd-comment-thread-renderer'})
    print('댓글정보 불러오기 시작')
    for i in range(len(comment0)):
        

        # 댓글
        comment = comment0[i].find('yt-formatted-string',{'id':'content-text','class':'style-scope ytd-comment-renderer'}).text
        # 댓글의 좋아요 수
        try:
            aa = comment0[i].find('span',{'id':'vote-count-left'}).text
            #정규표현식으로 숫자만 추출하는 것은 정규표현식에 대한 공부를 더 한 뒤 해결
            #re.findall('[0-9]',aa)
            #"".join(re.findall('[0-9]',aa)) -> 리스트 내부의 문자열의 합
            like_num = "".join(re.findall('[0-9]',aa))
            like_num = int(like_num)
        except:
            like_num = 0

        # 댓글 작성자 id    
        bb = comment0[i].find('a',{'id':'author-text'}).find('span').text
        youtube_id = "".join(re.findall('[가-힣0-9a-zA-Z]',bb))

        # 댓글 작성 날짜
        comment_date = comment0[i].find('yt-formatted-string',{'class':'published-time-text style-scope ytd-comment-renderer'}).text
        
        # 데이터프레임에 데이터 저장
        comment_data.loc[len(comment_data), :] = [youtube_id, comment, like_num, comment_date]

    comment_data.index = range(len(comment_data))
    # 파일 저장
    if not os.path.exists(keyword):
        os.makedirs(keyword)

    comment_data.to_csv(f'./data/{keyword}/{keyword}_{video_num}_comment_data.csv')
    print(f'{keyword}_{video_num}_comment_data.csv 저장완료')
    

In [6]:
df = pd.read_csv('./data/회사원A/회사원A_video_data.csv', index_col=0)

In [5]:
for video_num in range(len(df)):
    url = df.iloc[video_num+1,1]
    driver.get(url)

    #페이지 Open 후 기다리는 시간
    time.sleep(10)
    
    # 영상 좋아요 수
    temp = driver.find_element(By.XPATH, '//*[@id="segmented-like-button"]/ytd-toggle-button-renderer/yt-button-shape/button')
    good = temp.get_attribute('aria-label')
    good_num = re.findall(r'\d{1,3}(?:,\d{3})*', good)
    if good_num:
        good_num = good_num[0].replace(',', '')  # 쉼표 제거
        df.iloc[video_num, 5] = good_num


    driver.implicitly_wait(3)

    time.sleep(1.5)

    driver.execute_script('window.scrollTo(0, 800)') # 한번 스크롤
    time.sleep(3)

    last_height = driver.execute_script('return document.documentElement.scrollHeight') # 스크롤 전체 높이
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight / 10);")
    while True:
        driver.execute_script('window.scrollTo(0, document.documentElement.scrollHeight / 2);')
        time.sleep(3)
        driver.execute_script('window.scrollTo(0, document.documentElement.scrollHeight);') # 스크롤 다운
        time.sleep(3)

        new_height = driver.execute_script('return document.documentElement.scrollHeight') # 스크롤 다운 후 스크롤 높이 

        if new_height == last_height: # 댓글 마지막 페이지면 while문 벗어남
            break

        last_height = new_height
        time.sleep(1.5)
    print('스크롤 끝')
    time.sleep(5)   
    # 댓글 정보 불러오기
    comments_list = driver.find_elements(By.XPATH, '//*[@id="comment"]')

    comment_data = pd.DataFrame({'youtube_id':[],
                                'comment':[],
                                'like_num':[], 'comment_date':[]})
    html_s0 = driver.page_source
    html_s = BeautifulSoup(html_s0,'html.parser')

    comment0 = html_s.find_all('ytd-comment-renderer',{'class':'style-scope ytd-comment-thread-renderer'})
    print('댓글정보 불러오기 시작')
    for i in range(len(comment0)):
        

        # 댓글
        comment = comment0[i].find('yt-formatted-string',{'id':'content-text','class':'style-scope ytd-comment-renderer'}).text
        # 댓글의 좋아요 수
        try:
            aa = comment0[i].find('span',{'id':'vote-count-left'}).text
            #정규표현식으로 숫자만 추출하는 것은 정규표현식에 대한 공부를 더 한 뒤 해결
            #re.findall('[0-9]',aa)
            #"".join(re.findall('[0-9]',aa)) -> 리스트 내부의 문자열의 합
            like_num = "".join(re.findall('[0-9]',aa))
            like_num = int(like_num)
        except:
            like_num = 0

        # 댓글 작성자 id    
        bb = comment0[i].find('a',{'id':'author-text'}).find('span').text
        youtube_id = "".join(re.findall('[가-힣0-9a-zA-Z]',bb))

        # 댓글 작성 날짜
        comment_date = comment0[i].find('yt-formatted-string',{'class':'published-time-text style-scope ytd-comment-renderer'}).text
        
        # 데이터프레임에 데이터 저장
        comment_data.loc[len(comment_data), :] = [youtube_id, comment, like_num, comment_date]

    comment_data.index = range(len(comment_data))
    # 파일 저장


    comment_data.to_csv(f'./data/{youtuber}/{youtuber}_{video_num}_comment_data.csv')
    print(f'{youtuber}_{video_num}_comment_data.csv 저장완료')

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=116.0.5845.188)
Stacktrace:
	GetHandleVerifier [0x00007FF7D1CA52A2+57122]
	(No symbol) [0x00007FF7D1C1EA92]
	(No symbol) [0x00007FF7D1AEE3AB]
	(No symbol) [0x00007FF7D1ACE1B9]
	(No symbol) [0x00007FF7D1B4B417]
	(No symbol) [0x00007FF7D1B5E24F]
	(No symbol) [0x00007FF7D1B46DB3]
	(No symbol) [0x00007FF7D1B1D2B1]
	(No symbol) [0x00007FF7D1B1E494]
	GetHandleVerifier [0x00007FF7D1F4EF82+2849794]
	GetHandleVerifier [0x00007FF7D1FA1D24+3189156]
	GetHandleVerifier [0x00007FF7D1F9ACAF+3160367]
	GetHandleVerifier [0x00007FF7D1D36D06+653702]
	(No symbol) [0x00007FF7D1C2A208]
	(No symbol) [0x00007FF7D1C262C4]
	(No symbol) [0x00007FF7D1C263F6]
	(No symbol) [0x00007FF7D1C167A3]
	BaseThreadInitThunk [0x00007FFC2E757614+20]
	RtlUserThreadStart [0x00007FFC304026B1+33]


In [7]:
driver = webdriver.Chrome("chromedriver-win64/chromedriver.exe")

C:\Users\Mgyu\AppData\Local\Temp\ipykernel_16336\93324219.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver-win64/chromedriver.exe")


In [178]:
for video_num in range(len(df)):
    url = df.iloc[video_num,1]
    driver.get(url)

    #페이지 Open 후 기다리는 시간
    time.sleep(5)
    
    # 영상 좋아요 수
    temp = driver.find_element(By.XPATH, '//*[@id="segmented-like-button"]/ytd-toggle-button-renderer/yt-button-shape/button')
    good = temp.get_attribute('aria-label')
    good_num = re.findall(r'\d{1,3}(?:,\d{3})*', good)
    if good_num:
        good_num = good_num[0].replace(',', '')  # 쉼표 제거
        df.iloc[video_num, 5] = good_num



    #down the scroll
    body = driver.find_element(By.TAG_NAME, 'body')
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    # max size 50의 Queue 생성
    # 0.1sec * 50 = 5sec 동안 Scroll 업데이트가 없으면 스크롤 내리기 종료
    szQ = Queue(4)
    enqueue_count = 0
    time.sleep(2)
    i = 0
    while True:
        # Scroll 내리기
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        if i == 0:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight / 10);")
            time.sleep(3)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight / 2);")
            time.sleep(3) 
        # Scroll Height를 가져오는 주기
        time.sleep(5)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        # Queue가 꽉 차는 경우 스크롤 내리기 종료
        if(enqueue_count > szQ.maxsize):
            break
        
        # 첫 Loop 수행 (Queue가 비어있는 경우) 예외 처리
        if(szQ.isEmpty()) :
            szQ.enqueue(new_page_height)
            enqueue_count += 1
            
        # Queue에 가장 먼저 들어온 데이터와 새로 업데이트 된 Scroll Height를 비교함
        # 같으면 그대로 Enqueue, 다르면 Queue의 모든 Data를 Dequeue 후 새로운 Scroll Height를 Enqueue 함.    
        else :
            if(szQ.peek() == new_page_height) :
                szQ.enqueue(new_page_height)
                enqueue_count += 1
            else :
                szQ.enqueue(new_page_height)
                for z in range(enqueue_count) :
                    szQ.dequeue()
                enqueue_count = 1
        # time.sleep(3)
        if i > 10:
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(3)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight / 3);")
            time.sleep(3)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight * 2 / 3);")
            time.sleep(3)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            break
        else:
            i += 1

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=116.0.5845.188)
Stacktrace:
	GetHandleVerifier [0x00007FF6F41A52A2+57122]
	(No symbol) [0x00007FF6F411EA92]
	(No symbol) [0x00007FF6F3FEE3AB]
	(No symbol) [0x00007FF6F3FCE1B9]
	(No symbol) [0x00007FF6F404B417]
	(No symbol) [0x00007FF6F405E24F]
	(No symbol) [0x00007FF6F4046DB3]
	(No symbol) [0x00007FF6F401D2B1]
	(No symbol) [0x00007FF6F401E494]
	GetHandleVerifier [0x00007FF6F444EF82+2849794]
	GetHandleVerifier [0x00007FF6F44A1D24+3189156]
	GetHandleVerifier [0x00007FF6F449ACAF+3160367]
	GetHandleVerifier [0x00007FF6F4236D06+653702]
	(No symbol) [0x00007FF6F412A208]
	(No symbol) [0x00007FF6F41262C4]
	(No symbol) [0x00007FF6F41263F6]
	(No symbol) [0x00007FF6F41167A3]
	BaseThreadInitThunk [0x00007FF8DDF37614+20]
	RtlUserThreadStart [0x00007FF8DE0826B1+33]
